# Movie Recommender System

## Importing Libraries and Datasets

In [58]:
import pandas as pd
import numpy as np

In [59]:
movies = pd.read_csv('movies.csv')
credit = pd.read_csv('credits.csv')

In [60]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""space war""}, {""id"": 3388, ""name"": ""space colony""}, {""id"": 3679, ""name"": ""society""}, {""id"": 3801, ""name"": ""space travel""}, {""id"": 9685, ""name"": ""futuristic""}, {""id"": 9840, ""name"": ""romance""}, {""id"": 9882, ""name"": ""space""}, {""id"": 9951, ""name"": ""alien""}, {""id"": 10148, ""name"": ""tribe""}, {""id"": 10158, ""name"": ""alien planet""}, {""id"": 10987, ""name"": ""cgi""}, {""id"": 11399, ""name"": ""marine""}, {""id"": 13065, ""name"": ""soldier""}, {""id"": 14643, ""name"": ""battle""}, {""id"": 14720, ""name"": ""love affair""}, {""id"": 165431, ""name"": ""anti war""}, {""id"": 193554, ""name"": ""power relations""}, {""id"": 206690, ""name"": ""mind and soul""}, {""id"": 209714, ""name"": ""3d""}]",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporation"", ""id"": 306}, {""name"": ""Dune Entertainment"", ""id"": 444}, {""name"": ""Lightstorm Entertainment"", ""id"": 574}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [ ]:
credit.head(1)['crew'].values

## Data Preprocessing

### Merging two dataframes

In [62]:
movies = movies.merge(credit, on='title')

In [63]:
movies.shape

(4809, 23)

### Extracting Important Features

In [64]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

### Finding and removing Missing Data

In [65]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [66]:
movies.dropna(inplace=True) # dropna removes missing rows in pandas dataframe and inplace true ensures removal is applied # directly to the original DataFrame rather than returning a new, modified copy.


In [67]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

### Bringing columns like genre , keywords ,cast , crew in right format

In [68]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

* The current format is a list of dictionary we need to bring it in list like ["Action,"Adventure","SciFi"]

In [69]:
import ast # as list is in string format we need to convert it to simple list format using literal_eval of ast module

In [70]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name']) # append the value of ith name , value of name has key as genre
    return L

In [71]:
movies['genres'] = movies['genres'].apply(convert) # apply() method in pandas lets you apply a custom function across each row or column of a DataFrame.


In [72]:
movies.iloc[0].genres

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

* Doing Same for the keywords column

In [73]:
movies['keywords'] = movies['keywords'].apply(convert) # Applying same convert to keywords

* Converting Cast column same but here we only take top 3 cast , just modify convert function a bit to extracting first 3 dict in list giving us 1st three names

In [74]:
def convert_cast(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name']) # append the value of ith name , value of name has key as genre
            counter += 1
        else:
            break
    return L

In [75]:
movies['cast'] = movies['cast'].apply(convert_cast)

In [ ]:
movies.iloc[0].crew

* For crew we fetch only that dictionary where job name is director

In [77]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name']) # append the value of ith name , value of name has key as genre
            break
    return L

In [78]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [79]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, shipwreck, strong woman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercreditsstinger]","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE.","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi6, british secret service, united kingdom]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy.","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret identity, burglar, hostage drama, time bomb, gotham city, vigilante, cover-up, superhero, villainess, tragic hero, terrorism, destruction, catwoman, cat burglar, imax, flood, criminal underworld, batman]","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the mysterious and exotic planet of Barsoom (Mars) and reluctantly becomes embroiled in an epic conflict. It's a world on the brink of collapse, and Carter rediscovers his humanity when he realizes the survival of Barsoom and its people rests in his hands.","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel, princess, alien, steampunk, martian, escape, edgar rice burroughs, alien race, superhuman strength, mars civilization, sword and planet, 19th century, 3d]","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


* Converting Overview column to lists as well so it will be easy to concate overview,cast,crew,genre etc in single list

In [80]:
movies['overview'] = movies['overview'].apply(lambda x:x.split()) # This is an anonymous function that takes an input (x) and uses the built-in str.split() method to split the string into a list of its space-separated parts.


In [81]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marine, is, dispatched, to, the, moon, Pandora, on, a, unique, mission,, but, becomes, torn, between, following, orders, and, protecting, an, alien, civilization.]","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations, mind and soul, 3d]","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, dead,, has, come, back, to, life, and, is, headed, to, the, edge, of, the, Earth, with, Will, Turner, and, Elizabeth, Swann., But, nothing, is, quite, as, it, seems.]","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, shipwreck, strong woman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercreditsstinger]","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, sends, him, on, a, trail, to, uncover, a, sinister, organization., While, M, battles, political, forces, to, keep, the, secret, service, alive,, Bond, peels, back, the, layers, of, deceit, to, reveal, the, terrible, truth, behind, SPECTRE.]","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi6, british secret service, united kingdom]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney, Harvey, Dent,, Batman, assumes, responsibility, for, Dent's, crimes, to, protect, the, late, attorney's, reputation, and, is, subsequently, hunted, by, the, Gotham, City, Police, Department., Eight, years, later,, Batman, encounters, the, mysterious, Selina, Kyle, and, the, villainous, Bane,, a, new, terrorist, leader, who, overwhelms, Gotham's, finest., The, Dark, Knight, resurfaces, to, protect, a, city, that, has, branded, him, an, enemy.]","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret identity, burglar, hostage drama, time bomb, gotham city, vigilante, cover-up, superhero, villainess, tragic hero, terrorism, destruction, catwoman, cat burglar, imax, flood, criminal underworld, batman]","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, military, captain, who's, inexplicably, transported, to, the, mysterious, and, exotic, planet, of, Barsoom, (Mars), and, reluctantly, becomes, embroiled, in, an, epic, conflict., It's, a, world, on, the, brink, of, collapse,, and, Carter, rediscovers, his, humanity, when, he, realizes, the, survival, of, Barsoom, and, its, people, rests, in, his, hands.]","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel, princess, alien, steampunk, martian, escape, edgar rice burroughs, alien race, superhuman strength, mars civilization, sword and planet, 19th century, 3d]","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


### Remove space between words in genre , cast columns etc

* like Sam Worthington becomes SamWorthington as if space is there Sam and Worthington Will be Treated as different entity by Recommender though it is name of same person same for words like Science Fiction

In [82]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])


In [83]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marine, is, dispatched, to, the, moon, Pandora, on, a, unique, mission,, but, becomes, torn, between, following, orders, and, protecting, an, alien, civilization.]","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, society, spacetravel, futuristic, romance, space, alien, tribe, alienplanet, cgi, marine, soldier, battle, loveaffair, antiwar, powerrelations, mindandsoul, 3d]","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, dead,, has, come, back, to, life, and, is, headed, to, the, edge, of, the, Earth, with, Will, Turner, and, Elizabeth, Swann., But, nothing, is, quite, as, it, seems.]","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatradingcompany, loveofone'slife, traitor, shipwreck, strongwoman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercreditsstinger]","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, sends, him, on, a, trail, to, uncover, a, sinister, organization., While, M, battles, political, forces, to, keep, the, secret, service, alive,, Bond, peels, back, the, layers, of, deceit, to, reveal, the, terrible, truth, behind, SPECTRE.]","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, britishsecretservice, unitedkingdom]","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney, Harvey, Dent,, Batman, assumes, responsibility, for, Dent's, crimes, to, protect, the, late, attorney's, reputation, and, is, subsequently, hunted, by, the, Gotham, City, Police, Department., Eight, years, later,, Batman, encounters, the, mysterious, Selina, Kyle, and, the, villainous, Bane,, a, new, terrorist, leader, who, overwhelms, Gotham's, finest., The, Dark, Knight, resurfaces, to, protect, a, city, that, has, branded, him, an, enemy.]","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretidentity, burglar, hostagedrama, timebomb, gothamcity, vigilante, cover-up, superhero, villainess, tragichero, terrorism, destruction, catwoman, catburglar, imax, flood, criminalunderworld, batman]","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, military, captain, who's, inexplicably, transported, to, the, mysterious, and, exotic, planet, of, Barsoom, (Mars), and, reluctantly, becomes, embroiled, in, an, epic, conflict., It's, a, world, on, the, brink, of, collapse,, and, Carter, rediscovers, his, humanity, when, he, realizes, the, survival, of, Barsoom, and, its, people, rests, in, his, hands.]","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, princess, alien, steampunk, martian, escape, edgarriceburroughs, alienrace, superhumanstrength, marscivilization, swordandplanet, 19thcentury, 3d]","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


### Combining overview genre keywords cast crew into a single column tags and making a new dataframe with movie_id , title and tags

In [84]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [85]:
new_movies = movies[['movie_id','title','tags']]

In [86]:
new_movies.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marine, is, dispatched, to, the, moon, Pandora, on, a, unique, mission,, but, becomes, torn, between, following, orders, and, protecting, an, alien, civilization., Action, Adventure, Fantasy, ScienceFiction, cultureclash, future, spacewar, spacecolony, society, spacetravel, futuristic, romance, space, alien, tribe, alienplanet, cgi, marine, soldier, battle, loveaffair, antiwar, powerrelations, mindandsoul, 3d, SamWorthington, ZoeSaldana, SigourneyWeaver, JamesCameron]"
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, dead,, has, come, back, to, life, and, is, headed, to, the, edge, of, the, Earth, with, Will, Turner, and, Elizabeth, Swann., But, nothing, is, quite, as, it, seems., Adventure, Fantasy, Action, ocean, drugabuse, exoticisland, eastindiatradingcompany, loveofone'slife, traitor, shipwreck, strongwoman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercreditsstinger, JohnnyDepp, OrlandoBloom, KeiraKnightley, GoreVerbinski]"
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, sends, him, on, a, trail, to, uncover, a, sinister, organization., While, M, battles, political, forces, to, keep, the, secret, service, alive,, Bond, peels, back, the, layers, of, deceit, to, reveal, the, terrible, truth, behind, SPECTRE., Action, Adventure, Crime, spy, basedonnovel, secretagent, sequel, mi6, britishsecretservice, unitedkingdom, DanielCraig, ChristophWaltz, LéaSeydoux, SamMendes]"
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney, Harvey, Dent,, Batman, assumes, responsibility, for, Dent's, crimes, to, protect, the, late, attorney's, reputation, and, is, subsequently, hunted, by, the, Gotham, City, Police, Department., Eight, years, later,, Batman, encounters, the, mysterious, Selina, Kyle, and, the, villainous, Bane,, a, new, terrorist, leader, who, overwhelms, Gotham's, finest., The, Dark, Knight, resurfaces, to, protect, a, city, that, has, branded, him, an, enemy., Action, Crime, Drama, Thriller, dccomics, crimefighter, terrorist, secretidentity, burglar, hostagedrama, timebomb, gothamcity, vigilante, cover-up, superhero, villainess, tragichero, terrorism, destruction, catwoman, catburglar, imax, flood, criminalunderworld, batman, ChristianBale, MichaelCaine, GaryOldman, ChristopherNolan]"
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, military, captain, who's, inexplicably, transported, to, the, mysterious, and, exotic, planet, of, Barsoom, (Mars), and, reluctantly, becomes, embroiled, in, an, epic, conflict., It's, a, world, on, the, brink, of, collapse,, and, Carter, rediscovers, his, humanity, when, he, realizes, the, survival, of, Barsoom, and, its, people, rests, in, his, hands., Action, Adventure, ScienceFiction, basedonnovel, mars, medallion, spacetravel, princess, alien, steampunk, martian, escape, edgarriceburroughs, alienrace, superhumanstrength, marscivilization, swordandplanet, 19thcentury, 3d, TaylorKitsch, LynnCollins, SamanthaMorton, AndrewStanton]"


In [ ]:
new_movies['tags'] = new_movies['tags'].apply(lambda x:" ".join(x))
# if x is a list (or iterable) of strings, this function will concatenate all the strings in x into one string, with spaces in between each element. 1. It calls the join method on the string literal " ", which joins the elements of x using a space i.e " " as a separator.


In [88]:
new_movies.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron"
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems. Adventure Fantasy Action ocean drugabuse exoticisland eastindiatradingcompany loveofone'slife traitor shipwreck strongwoman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski"
2,206647,Spectre,"A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE. Action Adventure Crime spy basedonnovel secretagent sequel mi6 britishsecretservice unitedkingdom DanielCraig ChristophWaltz LéaSeydoux SamMendes"
3,49026,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy. Action Crime Drama Thriller dccomics crimefighter terrorist secretidentity burglar hostagedrama timebomb gothamcity vigilante cover-up superhero villainess tragichero terrorism destruction catwoman catburglar imax flood criminalunderworld batman ChristianBale MichaelCaine GaryOldman ChristopherNolan"
4,49529,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the mysterious and exotic planet of Barsoom (Mars) and reluctantly becomes embroiled in an epic conflict. It's a world on the brink of collapse, and Carter rediscovers his humanity when he realizes the survival of Barsoom and its people rests in his hands. Action Adventure ScienceFiction basedonnovel mars medallion spacetravel princess alien steampunk martian escape edgarriceburroughs alienrace superhumanstrength marscivilization swordandplanet 19thcentury 3d TaylorKitsch LynnCollins SamanthaMorton AndrewStanton"


In [ ]:
new_movies['tags'] = new_movies['tags'].apply(lambda x:x.lower())

In [90]:
new_movies['tags'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

## Vectorization

* the most frequent words have also included words like action,actions which are same but taken different due to plurals  , we use stemming to treat these words as same

In [91]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [92]:
def stem(text):
    y= []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_movies['tags'] = new_movies['tags'].apply(stem)

In [94]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english') # extract top 5000 most occuring words in tags and excluding stop_words

In [95]:
vectors = cv.fit_transform(new_movies['tags']).toarray() # it returns a sparse matrix we convert to numpy array

In [98]:
np.set_printoptions(threshold=np.inf)

In [ ]:
print(cv.get_feature_names_out())